# This file solve lasso using the `GradientDescentOptimizer` allocated with dataset API such as
`tf.data.Dataset.from_tensor_slices()`, `batch` and `batch_size`

Solving the problem
\begin{eqnarray}
\left[\begin{matrix}
3 & 0\\
1 & 2\\
0 & 0
\end{matrix}\right]
\left[\begin{matrix}
w_1\\
w_2
\end{matrix}\right]=
\left[\begin{matrix}
0\\
0\\
1
\end{matrix}\right],
\end{eqnarray}
Obviously, this equation has no accurate solution. So try to solve it using lasso, using some descent method in tensorflow:
$$
\mathop{\text{min}}\limits_{w}~\|Aw-b\|_2^2+\lambda \|w\|_1^2
$$
with
$$
A=
\left[\begin{matrix}
3 & 0\\
1 & 2\\
0 & 0
\end{matrix}\right],
b=
\left[\begin{matrix}
0\\
0\\
1
\end{matrix}\right],
$$
and with $\lambda$ given (for example, $\lambda=.001$).

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
learning_rate=0.01
training_epoch=100
display_step=20
batch_size=2

In [3]:
train_features=np.array([[3.0, 0.0], [1.0, 2.0], [0.0, 0.0]], dtype="float32")
train_labels=np.array([[0], [0], [1]], dtype="float32")

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
dataset = dataset.batch(batch_size)
iterator = dataset.make_initializable_iterator()
batch_data = iterator.get_next()

In [5]:
A=tf.placeholder("float32",[None, 2])
b=tf.placeholder("float32",[None, 1])
W=tf.Variable(np.random.randn(2), name="weight", dtype="float32")

In [6]:
predict=tf.multiply(A, W)
cost=tf.reduce_sum(tf.pow(predict-b, 2))

In [7]:
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # refresh for global variables
    for epoch in range(training_epoch):
        sess.run(iterator.initializer) # refresh for dataset reuse
        for k in range(4-batch_size):  # since tatal number of trainable examples is 3
            temp=sess.run(batch_data)
            sess.run(optimizer, feed_dict = {A: temp[0], b: temp[1]})
            if (epoch+1) % display_step == 0:
                print 'epoch=',epoch+1, 'cost=', sess.run(cost, feed_dict={A: temp[0], b: temp[1]}), 'weight=',sess.run(W)
                print 'training examples is:', (temp[0], temp[1])

epoch= 20 cost= 0.382492 weight= [0.00422857 0.3091574 ]
training examples is: (array([[3., 0.],
       [1., 2.]], dtype=float32), array([[0.],
       [0.]], dtype=float32))
epoch= 20 cost= 2.0 weight= [0.00422857 0.3091574 ]
training examples is: (array([[0., 0.]], dtype=float32), array([[1.]], dtype=float32))
epoch= 40 cost= 0.013612358 weight= [4.8752128e-05 5.8335952e-02]
training examples is: (array([[3., 0.],
       [1., 2.]], dtype=float32), array([[0.],
       [0.]], dtype=float32))
epoch= 40 cost= 2.0 weight= [4.8752128e-05 5.8335952e-02]
training examples is: (array([[0., 0.]], dtype=float32), array([[1.]], dtype=float32))
epoch= 60 cost= 0.00048466946 weight= [5.6207386e-07 1.1007605e-02]
training examples is: (array([[3., 0.],
       [1., 2.]], dtype=float32), array([[0.],
       [0.]], dtype=float32))
epoch= 60 cost= 2.0 weight= [5.6207386e-07 1.1007605e-02]
training examples is: (array([[0., 0.]], dtype=float32), array([[1.]], dtype=float32))
epoch= 80 cost= 1.7256747e-05

In [9]:
print np.dot(np.linalg.pinv(train_features), train_labels)
print np.dot(np.linalg.pinv(train_features), train_features)

[[0.]
 [0.]]
[[ 9.9999994e-01 -2.9802322e-08]
 [-5.9604645e-08  1.0000000e+00]]
